In [114]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain.tools.retriever import create_retriever_tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun
from langchain.agents import create_openai_tools_agent
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import AgentExecutor
from dotenv import load_dotenv

In [115]:
load_dotenv()
llm= ChatOpenAI(model='gpt-3.5-turbo', temperature = 0)

In [116]:
## wiki-tool
wiki_api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki_tool = WikipediaQueryRun(api_wrapper=wiki_api_wrapper)
wiki_tool.name

'wikipedia'

In [117]:
## web-page-parsing-tool - we are using https://docs.smith.langchain.com as example.
## https://python.langchain.com/v0.1/docs/use_cases/question_answering/conversational_retrieval_agents/
loader = WebBaseLoader('https://docs.smith.langchain.com/')
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vectordb = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vectordb.as_retriever()
langsmith_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)
langsmith_tool.name

'langsmith_search'

In [118]:
## Arxiv tool
## Arxiv: https://arxiv.org/   
##arXiv is a free distribution service and an open-access archive for nearly 2.4 million scholarly articles in the fields of physics, mathematics, computer science, quantitative biology, quantitative finance, statistics, electrical engineering and systems science, and economics. Materials on this site are not peer-reviewed by arXiv.
arxiv_api_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv_tool = ArxivQueryRun(api_wrapper=arxiv_api_wrapper)
arxiv_tool.name

'arxiv'

In [119]:
## create a prompt for the tool below.
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [120]:
## Agents
## https://python.langchain.com/v0.1/docs/modules/agents/
## The core idea of agents is to use a language model to choose a sequence of actions to take. In chains, a sequence of actions is hardcoded (in code).
## In agents, a language model is used as a reasoning engine to determine which actions to take and in which order.
tools = [wiki_tool, arxiv_tool, langsmith_tool]
# we got three tools, agent will search each tool for my qry.  if it does not find in wiki_too, it will try langsmith_tool, 
# if it does not find it in langsmith tool, will try Arxiv_tool.
agent=create_openai_tools_agent(llm=llm, tools=tools, prompt=prompt)

In [121]:
## Agent Executer
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose = True)
#response = agent_executor.invoke({'input':'tells me about LangSmith'}) # will pick info from LangSmith web page
#response = agent_executor.invoke({'input':'tells me about Machine Learning'})  # will pick info from Wiki
response = agent_executor.invoke({'input':'tells me what this paper 1605.08386 is all about'})   # will pick info from Arxiv
print(response['input'])
print(response['output'])



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1605.08386'}`


Published: 2016-05-26
Title: Heat-bath random walks with Markov bases
Authors: Caprice Stanley, Tobias Windisch
Summary: Graphs on lattice points are studied whose edges come from a finite set of
alloThe paper with identifier 1605.08386 is titled "Heat-bath random walks with Markov bases" by authors Caprice Stanley and Tobias Windisch. The paper discusses the study of graphs on lattice points where the edges are derived from a finite set of allocations.

> Finished chain.
tells me what this paper 1605.08386 is all about
The paper with identifier 1605.08386 is titled "Heat-bath random walks with Markov bases" by authors Caprice Stanley and Tobias Windisch. The paper discusses the study of graphs on lattice points where the edges are derived from a finite set of allocations.
